In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from ipypublish import nb_setup
import os
from os.path import join

# https://stackoverflow.com/a/39566040/11552622
rcparams = {
    'axes.titlesize':13,
    'axes.labelsize':9,
    'xtick.labelsize':8,
    'ytick.labelsize':8
}
plt = nb_setup.setup_matplotlib(output=('pdf','svg'), rcparams=rcparams)
pd = nb_setup.setup_pandas(escape_latex=False)
sns.set(style='whitegrid')

# Introduction

## Aim

Visualize and analyze sensitivity analysis data.

# Data check

## Distribution of sensitivity analysis variables

For each value of $\epsilon$, we generate four sets of simulations - one for each variable among $E_{max,lv}$, $E_{max,lv,0}$, $G_{E_{max,lv}}$, $k_{E,lv}$. In each set:

* 3 variables $P_i$ are set to the constant value $(P_{i,max} + P_{i,min})/2$, $i \in \{0,1,2,3\}$
* the last variable $P_j$ is sampled for each simulation according to the formula:

$$\frac{(P_{i,max} + P_{i,min})}{2} (1+\epsilon x)$$

* with $x$ a sample of the random variable $X \sim \mathcal{N}(0, 1)$, $i \in \{0,1,2,3\}$

In [ ]:
path = '/media/maousi/Data/tmp/sensitivity'
params = []
# For each value of epsilon
for folder in os.listdir(path):
    # Extract epsilon value
    epsilon = folder.split('_')[-1]
    # For each 0D model parameter
    dfepsilon = []
    for subfolder in os.listdir(join(path, folder)):
        # Extract parameter name
        param = subfolder[3:]
        # Read parameters.txt file
        pathtmp = join(path, folder, subfolder)
        dftmp = pd.read_csv(join(pathtmp, 'parameters.txt'))
        # Extract the column of the varying parameter
        dfepsilon.append(dftmp[param])
    
    # Merge the parameter values for a given epsilon
    dfepsilon = pd.DataFrame(dfepsilon).transpose()
    # Spefiy the value of epsilon
    dfepsilon['epsilon'] = epsilon
    params.append(dfepsilon)

# Merge everything
params = pd.concat(params, ignore_index=True)

In [ ]:
params.describe()

In [ ]:
param_dic = dict(zip(params, ['$E_{max,lv}$', '$E_{max,lv,0}$', '$G_{E_{max,lv}}$', '$k_{E,lv}$']))
param_dic

In [ ]:
summary = params.rename(columns=param_dic)
summary = summary.groupby('epsilon').describe().transpose()
summary = summary.loc[ (slice(None), ('min', '50%', 'max', 'std')), : ]
summary.rename(index={'50%':'mean'})

In [ ]:
names = [p for p in params.columns if p != 'epsilon']
names = list(zip(names, 
                 ['$E_{max,lv}$', '$E_{max,lv,0}$', '$G_{E_{max,lv}}$', '$k_{E,lv}$'], 
                 [(0.2, 2.95), (0.2, 2.392), (0.2, 0.475), (0.011, 0.014)]
                 ))
names

In [ ]:
epsilons = params.epsilon.unique()
epsilons

In [ ]:
#summary = 

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.ravel()

for (p, platex, (min_, max_)), ax in zip(names, axes):
    for epsilon in epsilons:
        sns.distplot(params.loc[params.epsilon == epsilon, p], 
                     ax=ax, hist=False, 
                     label=r'$\epsilon = ' + str(epsilon) + '$')
    ax.set_xlabel(platex)
    ax.set_ylabel('Density')
    ax.grid(alpha=.7)
    ax.axvline(min_, ls='--', color='grey')
    ax.axvline(max_, ls='--', color='grey')
        
plt.subplots_adjust(hspace=.35, wspace=.3)
plt.savefig('figs/sensitivity_var_distrib.eps')

Distribution of the variables that vary (i.e. not fixed) during the sensitivity analysis. Each curve estimates the gaussian kernel distribution of 100 simulation samples generated by fixing three parameters while the fourth one varies. Each subplots shows the distribution of the non-fixed variable for different values of epsilon.

# Summary tables

**MANUALLY IMPORT THE LATEX FILES HERE, AFTER USING NBPUBLISH**. 